[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%203/Contaminant_Model_Roots_Example.ipynb)

# Finding Time to Reach Contaminant Standards using Root Finding
## Bisection Method Application

This notebook demonstrates how we can reformulate a problem as a roots problem and then use the bisection method to find the solution. We'll revisit the contaminant decay model from Week 1 but solve a slightly different problem.

### Learning Objectives
- Understand how to reformulate problems as root-finding problems
- Apply the bisection method to environmental engineering scenarios
- Conduct parameter sensitivity analysis using numerical methods
- Visualize the relationship between model parameters and solutions

### The Contaminant Decay Model

We'll again use the two-component contaminant decay model:

$$p(t) = A_0 \cdot e^{-k_A \cdot t} + B_0 \cdot e^{-k_B \cdot t}$$

Where:
- $p(t)$ = total contaminant concentration (ppm) at time $t$
- $t$ = time (days)
- $A_0$ = initial concentration of contaminant A (ppm)
- $B_0$ = initial concentration of contaminant B (ppm)
- $k_A$ = decay rate of contaminant A (day$^{-1}$)
- $k_B$ = decay rate of contaminant B (day$^{-1}$)

### The Problem

In this notebook, we want to find the exact time $t$ when the contaminant concentration reaches 10 ppm for various values of $k_B$. This can be formulated as finding the roots of the equation:

$$p(t) - 10 = 0$$

By solving this equation for different values of $k_B$, we can understand how the remediation timeline depends on the decay rate of contaminant B.

## Setup

First, we'll import the necessary libraries for numerical computing and visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set plot style for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]  # Set default figure size

## Model Parameters

Let's define our base model parameters. We'll keep the initial concentrations and the decay rate of contaminant A constant, while we'll vary the decay rate of contaminant B to analyze its effect on the time to reach the regulatory standard.

In [ ]:
# Model parameters
A0 = 200      # Initial concentration of contaminant A (ppm)
B0 = 100      # Initial concentration of contaminant B (ppm)
kA = 1.0      # Decay rate of contaminant A (day^-1)

# Regulatory standard concentration
standard = 10  # ppm

## Bisection Method Implementation

First, let's implement the bisection method from Week 3. We'll modify it to accept parameters so we can pass in the current $k_B$ value for each calculation.

In [ ]:
def bisection(f, a, b, tol=1e-6, max_iter=100, *args):
    """
    Find a root of function f(x) in the interval [a, b] using the bisection method.
    
    Parameters:
    f : function
        The function for which we want to find the root
    a, b : float
        The interval [a, b] where we search for the root
    tol : float, optional
        Tolerance for stopping criterion (default: 1e-6)
    max_iter : int, optional
        Maximum number of iterations (default: 100)
    *args : additional parameters
        Additional parameters to pass to the function f
        
    Returns:
    float: The estimated root
    int: Number of iterations
    float: Final approximate relative error
    """
    # Evaluate function at interval endpoints
    fa = f(a, *args)
    fb = f(b, *args)
    
    # Check if the function changes sign in the interval
    if fa * fb > 0:
        raise ValueError(f"Function must have opposite signs at interval endpoints: f({a}) = {fa}, f({b}) = {fb}")
    
    # Initialize iteration counter and error
    iter_count = 0
    ea = tol + 1  # ensures loop runs at least once
    
    # Bisection loop
    while ea > tol and iter_count < max_iter:
        # Find midpoint
        c = (a + b) / 2
        fc = f(c, *args)
        
        # Calculate approximate relative error
        if c != 0:
            ea = abs((b - a) / (2 * c))
        
        # Check if we found the root exactly
        if fc == 0:
            return c, iter_count, 0
        
        # Update interval
        if fa * fc < 0:
            b = c
            fb = fc
        else:
            a = c
            fa = fc
        
        iter_count += 1
    
    # Return final midpoint, iterations and error
    return c, iter_count, ea

## Define the Target Function

Now, let's define the function whose root we want to find. For our problem, we want to find when $p(t) - 10 = 0$, which means when the total contaminant concentration equals the regulatory standard of 10 ppm.

In [ ]:
def contaminant_root_function(t, kB):
    """
    Function that equals zero when contaminant concentration reaches
    the regulatory standard (10 ppm).
    
    Parameters:
    t : float
        Time (days)
    kB : float
        Decay rate of contaminant B (day^-1)
        
    Returns:
    float: Difference between p(t) and the standard
    """
    # Calculate concentration at time t
    concentration = A0 * np.exp(-kA * t) + B0 * np.exp(-kB * t)
    
    # Return the difference between the concentration and the standard
    return concentration - standard

## Sensitivity Analysis

Now, let's perform a parameter sensitivity analysis by finding the time to reach the regulatory standard for different values of $k_B$.

In [ ]:
# Create a range of kB values from 0.5 to 4 with 15 points
kB_values = np.linspace(0.5, 4, 15)

# Lists to store results
times = []
iterations = []
errors = []

# For each kB value, find the time to reach the standard
for kB in kB_values:
    # Define search interval based on expected behavior
    # For smaller kB, it will take longer to reach the standard
    max_search_time = max(15, 10 / kB)  
    
    # Apply bisection method to find the root
    try:
        time, iter_count, error = bisection(contaminant_root_function, 0, max_search_time, 1e-6, 100, kB)
        times.append(time)
        iterations.append(iter_count)
        errors.append(error)
    except ValueError as e:
        print(f"Error for kB = {kB}: {e}")
        # If bisection fails, append None to keep arrays aligned
        times.append(None)
        iterations.append(None)
        errors.append(None)

## Visualize Results

Now, let's create a plot that shows how the time to reach the regulatory standard varies with the decay rate of contaminant B.

In [ ]:
# Filter out any None values
valid_indices = [i for i, t in enumerate(times) if t is not None]
valid_kB = [kB_values[i] for i in valid_indices]
valid_times = [times[i] for i in valid_indices]

# Create the plot
plt.figure(figsize=(12, 7))

# Plot time to reach standard vs. kB
plt.plot(valid_kB, valid_times, 'bo-', linewidth=2, markersize=8)

# Add labels and legend
plt.title('Time to Reach 10 ppm Regulatory Standard vs. $k_B$', fontsize=14)
plt.xlabel('$k_B$ (decay rate of contaminant B, day$^{-1}$)', fontsize=12)
plt.ylabel('Time (days)', fontsize=12)
plt.grid(True, alpha=0.3)

# Annotate the plot with model parameters
plt.text(0.05, 0.95, f'$A_0$ = {A0} ppm\n$B_0$ = {B0} ppm\n$k_A$ = {kA} day$^{-1}$\nStandard = {standard} ppm', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.show()

## Interpret the Results

From the plot, we can observe the relationship between the decay rate of contaminant B ($k_B$) and the time it takes for the total concentration to reach the regulatory standard of 10 ppm.

Key observations:

1. As $k_B$ increases, the time to reach the standard decreases. This makes sense because a higher decay rate means faster breakdown of the contaminant.

2. The relationship is non-linear - we see diminishing returns as $k_B$ increases. This is important for decision-making: increasing the decay rate (perhaps through active treatment) may be very beneficial up to a point, after which further increases yield minimal time savings.

3. The curve approaches an asymptote, suggesting that even with very high values of $k_B$, there's a minimum time required to reach the standard. This is because contaminant A has a fixed decay rate in our model, and it eventually becomes the limiting factor.

This type of analysis is valuable for environmental remediation planning, helping engineers decide whether active treatment to increase decay rates is worth the investment, or if natural attenuation (with lower decay rates) is sufficient given time constraints.

## Conclusion

In this notebook, we demonstrated how to:

1. Reformulate an environmental engineering problem as a root-finding problem
2. Use the bisection method with parameterization to solve for the roots
3. Perform a sensitivity analysis by varying a key parameter ($k_B$)
4. Visualize and interpret the relationship between the parameter and the solution

This approach can be applied to many other engineering problems where we need to find when a system reaches a specific state or threshold, and understand how parameters affect the timing.